# Project Title
### WORLD CITIES LAND TEMPERATURE ETL & DATA WAREHOUSE

#### Project Summary
ETL Pipeline to Data Warehouse in AWS Redshift 

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
from helpers.get_kaggle_data import kaggle_download
from helpers.upload_to_s3 import upload_file
from helpers.data_quality_checks import data_quality_check
from helpers.sql import sql_create_tables, staging_insert, sql_drop_tables, sql_insert_tables
import pandas as pd
import os
import psycopg2
from configparser import ConfigParser

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

A Data Warehouse will be created to store clean organize data that will be accesable to multiple users on a AWS Redshift cluster. 

The ETL Pipeline will work as described below:

- A Kaggle API will be used to get and download the data
- Pandas will be used to explore and clean the data
- Dataframe will be saved as CSV file and uploaded to AWS s3 bucket
- Data will be imported to AWS Redshift
- Data quality tests will be carried out to make sure the data exists and is available 

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

In [ ]:
# Download Data from Kaggle API
# You must have an account with Kaggle
kaggle_download('berkeleyearth/climate-change-earth-surface-temperature-data')

In [ ]:
# Read the data
directory = os.path.realpath("..") + '/global-land-temp-dw/'
tables = ['GlobalLandTemperaturesByMajorCity.csv', 'GlobalLandTemperaturesByCity.csv']

df_major_city = pd.read_csv(directory + "data/" + tables[0])
df_city = pd.read_csv(directory + "data/" + tables[1])

In [ ]:
print("DF for info major cities")
df_major_city.info()

print("\nDF info for cities")
df_city.info()

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [ ]:
# Null/Missing Values
df_city.isnull().sum()

print("Null/Missing Values for major cities")
print(df_major_city.isnull().sum())

print("\nNull/Missing Values for cities")
print(df_city.isnull().sum())

In [ ]:
# Drop NA/missing values This data requires all values to be available
def drop_na(pd_df):

    print(f'Number of rows before removing NA: {pd_df.shape[0]:,}')

    pd_df.dropna(axis=0, inplace=True)

    print(f'Number of rows after removing NA: {pd_df.shape[0]:,}')


drop_na(df_major_city)
drop_na(df_city)


In [ ]:
# Add column for Boolean Major city
df_major_city = df_major_city.assign(major_city='true')
df_city = df_city.assign(major_city='false')



In [ ]:
# Concatenate both datasets
dframes = [df_major_city, df_city]
df = pd.concat(dframes, ignore_index=True)

In [ ]:
df.head()

In [ ]:
# Sort data by year and reset index
df.sort_values(by='dt',inplace=True)

df.reset_index(drop=True, inplace=True)

In [ ]:
df.head(1000)


In [ ]:
# Confirm Data types are as expected
df.dtypes

In [ ]:
# Save data to CSV to load into SQL redshift
df.to_csv(directory + "data/dataset.csv")

In [ ]:
# Get Bucket Name from configurations 
config = ConfigParser()
config.read('DW.cfg')
file_log = config.get('S3', 'log_data')

upload_file(directory + "data/dataset.csv", file_log, "logs/log_data.csv")


### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Star Schema

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Get Database Variables
config = ConfigParser()
config.read('DW.cfg')

print("{}, {}, {}, {}, {}".format(*config['CLUSTER'].values()))


# Connect to redshift database
conn = psycopg2.connect("""host={} 
                           dbname={} 
                           user={} 
                           password={}
                           port={}"""\
                           .format(*config['CLUSTER'].values()    
))

# conn.set_session(autocommit=True)
cur = conn.cursor()


In [ ]:
# Drop all table if exists
for query in sql_drop_tables:
    cur.execute(query)
    conn.commit()

In [ ]:
# Create tables
for query in sql_create_tables:
    cur.execute(query)
    conn.commit()


In [ ]:
# Copy data to Staging
s3_bucket = "s3://{}/logs/".format(config.get('S3','log_data'))

sql_query = staging_insert.format(
    s3_bucket,
    config.get('IAM_ROLE','ARN'),
    config.get('CLUSTER','dwh_region')
    )
# print(sql_query)
cur.execute(sql_query)
conn.commit()

In [ ]:
# Insert data from staging to facts and dimensions
for query in sql_insert_tables:
    # print(query)
    cur.execute(query)
    conn.commit()

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
tables = ['readings_by_city', 'cities', 'time']

for table in tables:

    data_quality_check(cur, table)

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

In [ ]:
data_dictionary = [
    {   
        'table':'Dimension cities',
        'field name': 'city_id',
        'data type': 'BIGINT IDENTITY(0,1)',
        'data format': 'NNNNNNN',
        'field size': 'VARIABLE',
        'description':'Primary Key Index number for each row',
        'example':'03'
    },
    {
        'table':'Dimension cities',
        'field name': 'city',
        'data type': 'TEXT',
        'data format': '',
        'field size': 'VARIABLE',
        'description':'city name, comes from staging_events',
        'example':'Barcelona'
    },
    {
        'table':'Dimension cities',
        'field name': 'country',
        'data type': 'TEXT',
        'data format': '',
        'field size': 'VARIABLE',
        'description':'country name, comes from staging_events',
        'example':'Venezuela'
    },
    {
        'table':'Dimension cities',
        'field name': 'latitude',
        'data type': 'TEXT',
        'data format': 'NN.NT',
        'field size': 'VARIABLE',
        'description':'city latitude, comes from staging_events',
        'example':'12.5N'
    },
    {
        'table':'Dimension cities',
        'field name': 'longitude',
        'data type': 'TEXT',
        'data format': 'NN.NT',
        'field size': 'VARIABLE',
        'description':'city longitude, comes from staging_events',
        'example':'12.5E'
    },
    {
        'table':'Dimension cities',
        'field name': 'major_city',
        'data type': 'BOOLEAN',
        'data format': 'true/false',
        'field size': '5',
        'description':'Boolean that identifies if city is major or not from staging_events',
        'example':'true'
    },
    {
        'table':'Dimension time',
        'field name': 'dt',
        'data type': 'DATE',
        'data format': 'YYYY-MM-DD',
        'field size': '10',
        'description':'date of temperature measure, comes from staging_events',
        'example':'2018-02-07'
    },
    {
        'table':'Dimension time',
        'field name': 'day',
        'data type': 'SMALLINT',
        'data format': 'NN',
        'field size': '2',
        'description':'day of temperature measure, comes from column dt',
        'example':'07'
    },
    {
        'table':'Dimension time',
        'field name': 'month',
        'data type': 'SMALLINT',
        'data format': 'NN',
        'field size': '2',
        'description':'month of temperature measure, comes from column dt',
        'example':'02'
    },
    {
        'table':'Dimension time',
        'field name': 'week',
        'data type': 'SMALLINT',
        'data format': 'NN',
        'field size': '2',
        'description':'week of temperature measure, comes from column dt',
        'example':'02'
    },
    {
        'table':'Dimension time',
        'field name': 'weekday',
        'data type': 'SMALLINT',
        'data format': 'NN',
        'field size': '2',
        'description':'week day number of temperature measure, comes from column dt',
        'example':'02'
    },
    {
        'table':'Dimension time',
        'field name': 'year',
        'data type': 'SMALLINT',
        'data format': 'NNNN',
        'field size': '4',
        'description':'year of temperature measure, comes from column dt',
        'example':'2018'
    },
    {
        'table':'Fact readings_by_city',
        'field name': 'by_city_id',
        'data type': 'BIGINT IDENTITY(0,1)',
        'data format': 'NNNNNN',
        'field size': 'VARIABLE',
        'description':'Primary Key Index number for each row',
        'example':'2'
    },
    {
        'table':'Fact readings_by_city',
        'field name': 'city_id',
        'data type': 'NUMERIC',
        'data format': 'NNNNNN',
        'field size': 'VARIABLE',
        'description':'Foreign Key Index number from cities table',
        'example':'4525'
    },
    {
        'table':'Fact readings_by_city',
        'field name': 'avg_temp',
        'data type': 'NUMERIC(7,3)',
        'data format': 'NNNN.NNN',
        'field size': '7',
        'description':'temperature reading comes from staging_events',
        'example':'25.025'
    },
    {
        'table':'Fact readings_by_city',
        'field name': 'avg_temp_uncertainty',
        'data type': 'NUMERIC(7,3)',
        'data format': 'NNNN.NNN',
        'field size': '7',
        'description':'temperature uncertainty reading comes from staging_events',
        'example':'1.025'
    },
    {
        'table':'Fact readings_by_city',
        'field name': 'date',
        'data type': 'DATE',
        'data format': 'YYYY-MM-DD',
        'field size': '7',
        'description':'date of temperature measure, comes from staging_events',
        'example':'2018-02-07'
    },
    {
        'table':'Fact readings_by_city',
        'field name': 'major_city',
        'data type': 'BOLEAN',
        'data format': '',
        'field size': '5',
        'description':'Boolean that identifies if city is major or not from staging_events',
        'example':'true'
    },
    
    
       
]

In [ ]:
data_dic_df = pd.DataFrame(data_dictionary)

data_dic_df.to_csv(directory + "media/data_dict.csv")

data_dic_df


#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.